# Distributed data parallel MNIST training with TensorFlow2 and SMDataParallel

SMDataParallel is a new capability in Amazon SageMaker to train deep learning models faster and cheaper. SMDataParallel is a distributed data parallel training framework for TensorFlow2, PyTorch and MXNet.  

This notebook example shows how to use SMDataParallel with TensorFlow2 in SageMaker using MNIST dataset.

For more information:
1. [TensorFlow in SageMaker](https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/using_tf.html)
2. [SMDataParallelTensorFlow API Specification] < LINK TO BE ADDED >
3. [Getting started with SMDataParallel on SageMaker] < LINK TO BE ADDED >

**NOTE:** This example requires SageMaker Python SDK v2.X.

### Dataset
This example uses the MNIST dataset. MNIST is a widely used dataset for handwritten digit classification. It consists of 70,000 labeled 28x28 pixel grayscale images of handwritten digits. The dataset is split into 60,000 training images and 10,000 test images. There are 10 classes (one for each of the 10 digits).

### SageMaker execution roles

The IAM role arn used to give training and hosting access to your data. See the [Amazon SageMaker Roles](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html) for how to create these. Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the sagemaker.get_execution_role() with the appropriate full IAM role arn string(s).


In [5]:
!pip install sagemaker==2.19.0

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/tensorflow_p36/bin/python -m pip install --upgrade pip' command.


In [6]:
import sagemaker

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

## Model training with SMDataParallel

### Training script

The `train_tensorflow_smdataparallel_mnist.py` script provides the code you need for training a SageMaker model using SMDataParallel's `DistributedGradientTape`. The training script is very similar to a TensorFlow2 training script you might run outside of SageMaker, but modified to run with SMDataParallel. SMDataParallel's TensorFlow client provides an alternative to native `DistributedGradientTape`. For details about how to use SMDataParallel in your native TF2 script, see the Getting started with SMDataParallel tutorial.


In [7]:
!pygmentize code/train_tensorflow_smdataparallel_mnist.py

# Licensed to the Apache Software Foundation (ASF) under one
# or more contributor license agreements.  See the NOTICE file
# distributed with this work for additional information
# regarding copyright ownership.  The ASF licenses this file
# to you under the Apache License, Version 2.0 (the
# "License"); you may not use this file except in compliance
# with the License.  You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing,
# software distributed under the License is distributed on an
# "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
# KIND, either express or implied.  See the License for the
# specific language governing permissions and limitations
# under the License.

import tensorflow as tf

# Import SMDataParallel TensorFlow2 Modules
import smdistributed.dataparallel.tensorflow as dist

import os

tf.random.set_seed(42)


# SMDataParallel: Initialize
dist.init()

gpus = tf.con

### SageMaker TensorFlow Estimator function options


In the following code block, you can update the estimator function to use a different instance type, instance count, and distrubtion strategy. You're also passing in the training script you reviewed in the previous cell.

**Instance types**

SMDataParallel supports model training on SageMaker with the following instance types only:
1. ml.p3.16xlarge
1. ml.p3dn.24xlarge [Recommended]
1. ml.p4d.24xlarge [Recommended]

**Instance count**

To get the best performance and the most out of SMDataParallel, you should use at least 2 instances, but you can also use 1 for testing this example.

**Distribution strategy**

Note that to use DDP mode, you update the the `distribution` strategy, and set it to use `smdistributed dataparallel`. 

In [10]:
from sagemaker.tensorflow import TensorFlow
estimator = TensorFlow(
                        base_job_name='tensorflow2-smdataparallel-mnist',
                        source_dir='code',
                        entry_point='train_tensorflow_smdataparallel_mnist.py',
                        role=role,
                        py_version='py37',
                        framework_version='2.3.1',
                        # For training with multinode distributed training, set this count. Example: 2
                        instance_count=2,
                        # For training with p3dn instance use - ml.p3dn.24xlarge
                        instance_type= 'ml.p3.16xlarge',
                        sagemaker_session=sagemaker_session,
                        # Training using SMDataParallel Distributed Training Framework
                        distribution={'smdistributed':{
                                            'dataparallel':{
                                                    'enabled': True
                                             }
                                      }}
                        )

In [11]:
estimator.fit()

2020-12-10 20:14:09 Starting - Starting the training job...
2020-12-10 20:14:32 Starting - Launching requested ML instancesProfilerReport-1607631248: InProgress
.........
2020-12-10 20:15:57 Starting - Preparing the instances for training.........
2020-12-10 20:17:38 Downloading - Downloading input data...
2020-12-10 20:17:54 Training - Downloading the training image...........2020-12-10 20:19:46.916346: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2020-12-10 20:19:46.927920: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2020-12-10 20:19:47.302436: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.11.0
2020-12-10 20:19:47.429051: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2


2020-12-10 20:19:56 Training - Training image download completed. Training in progress.[1,0]<stdout>:algo-1:167:167 [0] NCCL INFO Bootstrap : Using [0]eth0:10.2.234.77<0>
[1,0]<stdout>:algo-1:167:167 [0] NCCL INFO NET/Plugin : No plugin found (libnccl-net.so), using internal implementation
[1,8]<stdout>:algo-2:176:176 [0] NCCL INFO Bootstrap : Using [0]eth0:10.2.228.56<0>
[1,8]<stdout>:algo-2:176:176 [0] NCCL INFO NET/Plugin : No plugin found (libnccl-net.so), using internal implementation
[1,0]<stdout>:
[1,0]<stdout>:algo-1:167:167 [0] misc/ibvwrap.cc:63 NCCL WARN Failed to open libibverbs.so[.1]
[1,8]<stdout>:
[1,8]<stdout>:algo-2:176:176 [0] misc/ibvwrap.cc:63 NCCL WARN Failed to open libibverbs.so[.1]
[1,0]<stdout>:algo-1:167:167 [0] NCCL INFO NET/Socket : Using [0]eth0:10.2.234.77<0>
[1,0]<stdout>:algo-1:167:167 [0] NCCL INFO Using network Socket
[1,8]<stdout>:algo-2:176:176 [0] NCCL INFO NET/Socket : Using [0]eth0:10.2.228.56<0>
[1,8]<stdout>:algo-2:176:176 [0] NCCL INFO Using n

[1,0]<stdout>:algo-1:167:1400 [0] NCCL INFO Launch mode Parallel
[1,0]<stdout>:Step #0#011Loss: 2.306592
[1,0]<stdout>:algo-1:167:1392 [0] NCCL INFO Launch mode Parallel
[1,8]<stdout>:algo-2:176:1389 [0] NCCL INFO Launch mode Parallel
[1,0]<stdout>:Step #50#011Loss: 0.265049
[1,0]<stdout>:Step #100#011Loss: 0.245126
[1,0]<stdout>:Step #150#011Loss: 0.153079
[1,0]<stdout>:Step #200#011Loss: 0.214459
[1,0]<stdout>:Step #250#011Loss: 0.087619
[1,0]<stdout>:Step #300#011Loss: 0.077063
[1,0]<stdout>:Step #350#011Loss: 0.049630
[1,0]<stdout>:Step #400#011Loss: 0.181920
[1,0]<stdout>:Step #450#011Loss: 0.114983
[1,0]<stdout>:Step #500#011Loss: 0.183502
[1,0]<stdout>:Step #550#011Loss: 0.066536
[1,0]<stdout>:Step #600#011Loss: 0.089528
2020-12-10 20:20:27,057 sagemaker-training-toolkit INFO     Orted process exited
2020-12-10 20:20:27,036 sagemaker-training-toolkit INFO     Reporting training SUCCESS
2020-12-10 20:20:57,087 sagemaker-training-toolkit INFO     MPI process finished.
2020-12-10 2

## Next steps

Now that you have a trained model, you can deploy an endpoint to host the model. After you deploy the endpoint, you can then test it with inference requests. The following cell will store the model_data variable to be used with the inference notebook.

In [12]:
model_data = estimator.model_data
print("Storing {} as model_data".format(model_data))
%store model_data

Storing s3://sagemaker-us-east-1-835319576252/tensorflow2-smdataparallel-mnist-2020-12-10-20-14-08-681/output/model.tar.gz as model_data
Stored 'model_data' (str)
